In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = (SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.").getOrCreate())

In [ ]:
spark.sparkContext.setLogLevel("WARN")

In [ ]:
spark?

In [ ]:
book = spark.read.text("./1342-0.txt")

In [ ]:
# column_1 = book.select(book["value"])
# column_2 = book.select(book.value)

In [ ]:
# column_1.show(10, truncate = 70)
# column_2.show(10, truncate = 70)

In [ ]:
# book.show(15, truncate=100)

In [ ]:
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

In [ ]:
# import pyspark.sql.functions as F

# used functions:
#     1. col
#     2. explode
#     3. lower
#     4. split
#     5. regexp_extract

In [ ]:
lines = book.select(split(col("value"), " ").alias("line"))

In [ ]:
#lines = book.select(split(book.value, " ").alias("line"))

In [ ]:
#lines = book.select(split(book.value, " ").alias("line"))

In [ ]:
lines.show(10)

In [ ]:
words = lines.select(explode(col("line")).alias("book"))

In [ ]:
words.show(5)

In [ ]:
# convert to lowercase

lower_words = words.select(lower(col("book")).alias("lower_word"))

In [ ]:
lower_words.show(15)

In [ ]:
#  remove characters and spaces

clean_words = lower_words.select(regexp_extract(col("lower_word"), "[a-z]+", 0).alias("clean_word"))

remove_spaces = clean_words.filter(col("clean_word") != "")

In [ ]:
remove_spaces.show(15)

In [ ]:
removed_spaces = clean_words.filter(col("clean_word") != "")

In [ ]:
removed_spaces.show(15)

In [ ]:
grouped_words = removed_spaces.groupby(col("clean_word")).count()

In [ ]:
grouped_words.show()

In [ ]:
grouped_words.orderBy(col("count").desc()).show(5)

In [ ]:
results = grouped_words.orderBy(col("count"))

In [ ]:
results.show(5)

In [ ]:
# out_path = "./data/simple_count_single_partition.csv"

# results.write.format('csv').save("./data/my_file.csv")

# results.toPandas().to_csv('l2.csv')

In [ ]:
results.write.format('csv').save("./my_file.csv")

In [ ]:
book.coalesce(1).write.option("header","true").format("csv").save("./data.csv")

In [ ]:
RDDread.count()

In [ ]:
results.rdd.getNumPartitions()

In [ ]:
rdd.getNumPartitions()